<div class="alert alert-info">
<u><strong>Authors:</strong></u> <b>Ahmed Mukhtar</b> (ahmed.mukhtar@mail.polimi.it) and <b>Ahmed Yassin</b> (ahmedmohamed1@mail.polimi.it) - 2023 - Politecnico di Milano, Italy <br>
</div>

# Computation of the monthly 90th and 10th percentile indices

This Notebook computes the time series completeness of each Netatmo station for a specified year and month. Then, it computes the 90th and 10th percentiles of the time series noramlised with respect to the 90th and 10th percentiles of the average time series.

In [ ]:
import os
import folium
import pandas as pd
import seaborn as sns
import geopandas as gpd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from shapely.geometry import Point
import plotly.graph_objects as go
import calendar
%load_ext autoreload

In [ ]:
year_w = widgets.Dropdown(
    options = [i for i in range(2014, 2024)],
    value = 2023,
    description = 'Year:',
    disabled = False,
    layout = {'width': 'max-content'},
    style = {'description_width': 'initial'}
)
year_w

In [ ]:
year = year_w.value

In [ ]:
month_w = widgets.Dropdown(
    options = [i for i in range(1,13)],
    value = 1,
    description = 'Month:',
    disabled = False,
    layout = {'width': 'max-content'},
    style = {'description_width': 'initial'}
)
month_w

In [ ]:
month = month_w.value

In [ ]:
netatmo_out_path = 'Netatmo_csv_files/'

## 1. Time series completeness

### Hourly check

In [ ]:
# Load original data
netatmo_original = pd.read_csv(netatmo_out_path + 'temp_Net_milan_%s-%s_clip.csv' % (year, month), skiprows=0)
# Load cleaned data
netatmo_cleaned = pd.read_csv(netatmo_out_path + 'temp_Net_milan_%s-%s_filtered.csv' % (year, month), skiprows=0)

**Note**: `temp_net_df1` is the original data, `temp_net_df2` is cleaned data.

In [ ]:
def data_time_series_hourly(temp_net_df1, temp_net_df2, month, year, threshold):
    
    # Columns to be used
    columns = ['module_id', 'device_id', 'lat', 'long', 'timezone', 'country', 'altitude', 'city', 'street', 'geometry']

    # Group temp_net_df1 by module_id and count observations
    df1_n_obs = temp_net_df1.groupby('module_id').size().reset_index(name='n.obs(original)')
    # Merge with original data (netatmo_original)
    df1 = pd.merge(df1_n_obs, temp_net_df1.drop_duplicates('module_id')[columns], on='module_id', how='left')

    # Group by module_id and count observations
    df2_n_obs = temp_net_df2.groupby('module_id').size().reset_index(name='n.obs(cleaned)')
    # Merge with cleaned data (netatmo_cleaned)
    df2 = pd.merge(df2_n_obs, temp_net_df2.drop_duplicates('module_id')[columns], on='module_id', how='left')

    # Merge cleaned data with original observation count
    missing_values_df = pd.merge(df2, df1[['module_id', 'n.obs(original)']], on='module_id', how='left')

    # Calculate removed observations
    missing_values_df['removed.Obs'] = missing_values_df['n.obs(original)'] - missing_values_df['n.obs(cleaned)']

    # Determine total hours in the month
    if month in [1, 3, 5, 7, 8, 10, 12]:  # 31 days in January, March, May, July, August, October, December
        total_hours = 31 * 24
    elif month in [4, 6, 9, 11]:  # 30 days in April, June, September, November
        total_hours = 30 * 24
    elif month == 2:  # February
        # Check for leap year
        if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
            total_hours = 29 * 24  # 29 days in February
        else:
            total_hours = 28 * 24  # 28 days in February

    # check for observations time_serries percentage of each sensor
    missing_values_df['Time-Serries%'] = (missing_values_df['n.obs(cleaned)'] / total_hours)*100

    # Select relevant columns
    result_df = missing_values_df[['module_id', 'n.obs(original)', 'n.obs(cleaned)', 'removed.Obs', 'Time-Serries%']]
    
    # Filter rows where 'remaining.Obs' is equal to 1 (compete)
    complete_time_serries = result_df[result_df['Time-Serries%'] >= threshold]
    
    ### CREAT COMBINED PLOT ###
    month_name = calendar.month_name[month]
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    # Plot the histogram on the first subplot
    sns.histplot(result_df['Time-Serries%'], bins=20, kde=True, ax=ax1)
    ax1.set_title(f'Distribution of time series completeness - {month_name} {year}', fontsize=20)
    ax1.set_xlabel('Percentage of completeness', fontsize=18)
    ax1.set_ylabel('Number of stations', fontsize=18)
    ax1.tick_params(axis='both', which='major', labelsize=16)
    
    # Plot the pie chart on the second subplot
    complete_TS = len(complete_time_serries)
    incomplete_TS = len(result_df)-len(complete_time_serries)
    
    ax2.pie([complete_TS, incomplete_TS], labels=[f'Completeness >{threshold}% ({complete_TS})', f'Completeness <{threshold}% ({incomplete_TS})'],
        autopct='%1.1f%%', startangle=130, colors=['red','#66b3ff'], textprops={'fontsize': 16})
    ax2.set_title(f'Proportion of time series completeness', fontsize=20)
    
    plt.tight_layout()

    # Save the plot as an image (e.g., PNG, JPG, PDF)
    hist_path = "10th_90th_Percentile_Index/time_series_histograms/"
    plt.savefig(hist_path + f'stations_Time-Series_{year}_{month}.png')

    plt.show()

    return result_df, complete_time_serries, plt

In [ ]:
threshold_w = widgets.Dropdown(
    options = [i for i in range(1, 101)],
    value = 80,
    description = 'threshold:',
    disabled = False,
    layout = {'width': 'max-content'},
    style = {'description_width': 'initial'}
)
threshold_w

In [ ]:
threshold = threshold_w.value

In [ ]:
result_df, complete_time_serries, plt = data_time_series_hourly(netatmo_original, netatmo_cleaned, month, year, threshold)

In [ ]:
#result_df

In [ ]:
#complete_time_serries

In [ ]:
netatmo_cleaned['time'] = pd.to_datetime(netatmo_cleaned['time'], errors='coerce')
grouped = complete_time_serries.groupby('module_id')
all_complete_modules = set()
for module_id, group in grouped:
    all_complete_modules.add(module_id)
if all_complete_modules:
    complete_modules = netatmo_cleaned[netatmo_cleaned['module_id'].isin(all_complete_modules)]
    percentiles_per_module = complete_modules.groupby('module_id')['Temperature'].quantile([0.9, 0.1]).unstack()
    hourly_averages = complete_modules.groupby(pd.Grouper(key='time', freq='h'))['Temperature'].mean()
    average_90th_percentile = hourly_averages.quantile(0.9)
    average_10th_percentile = hourly_averages.quantile(0.1)
    
    index_90th = percentiles_per_module[0.9] / average_90th_percentile
    index_10th = percentiles_per_module[0.1] / average_10th_percentile
    
    indices = complete_modules.groupby('module_id').agg({
        'Temperature': 'mean',
        'device_id': 'first',
        'lat': 'first',
        'long': 'first',
        'timezone': 'first',
        'country': 'first',
        'altitude': 'first',
        'city': 'first',
        'street': 'first',
        'geometry': 'first'
    }).reset_index()
    
    indices['90th Percentile Index'] = indices['module_id'].map(index_90th)
    indices['10th Percentile Index'] = indices['module_id'].map(index_10th)
    
indices = indices[['module_id', 'Temperature', '90th Percentile Index', '10th Percentile Index',
                               'lat', 'long', 'city', 'street', 'altitude', 'geometry']]

path = '10th_90th_Percentile_Index/'
indices.to_csv(path + f'indices_{year}_{month}.csv')

In [ ]:
print("Number of stations:", netatmo_cleaned['module_id'].nunique())
print("Stations with complete time-series (> threshold):", len(complete_time_serries))
print("Stations with incomplete time-series (< threshold):", (netatmo_cleaned['module_id'].nunique()-len(complete_time_serries)))

## 2. Computation of normalised 10th and 90th percentiles

In [ ]:
#indices

In [ ]:
def plot_index_map(selected_df, Index):
    # Create 'geometry' column as Point objects
    geometry_array = [Point(xy) for xy in zip(selected_df['long'], selected_df['lat'])]
    # Create a GeoDataFrame
    selected_gdf = gpd.GeoDataFrame(selected_df, geometry=geometry_array, crs='EPSG:4326')

    aoi_gdf = gpd.read_file('CMM.gpkg')
    m = aoi_gdf.explore(
        marker_kwds=dict(radius=10, fill=True),
        tooltip_kwds=dict(labels=False),
        tooltip=False,
        popup=False,
        highlight=False,
        name="cmm"
    )

    selected_gdf.explore(
        m=m,
        column= Index,
        tooltip=['module_id', 'city', 'street','altitude', Index],
        popup=True,
        tiles="CartoDB positron",
        #marker_kwds=dict(radius=4, fill=False, color='red'),
        marker_kwds=dict(radius=4, fill=True, color='red_color_ramp'),
        cmap='Spectral_r'
    )
    
    # Add north arrow
    north_arrow_html = '''
    <div style="position: fixed; top: 50px; right: 35px; width: 52px; height: 70px; 
                background-color: white; border:2px solid black; z-index:9999; font-size:15px;">
        <img src="https://th.bing.com/th/id/R.c66b4aa98a0cbcf2c9c62da5819e528a?rik=hnWDHcMoO8Hnhw&riu=http%3a%2f%2fpluspng.com%2fimg-png%2ffree-png-north-arrow-big-image-png-1659.png&ehk=KTDoWmlfwvZGkF3%2f3F9JY1ltV6DkLkrkHog%2fpXQiyqc%3d&risl=&pid=ImgRaw&r=0" 
             style="width:50px; height:60px;">
    </div>
    '''
    m.get_root().html.add_child(folium.Element(north_arrow_html))

    return m

### 90th percentile index

In [ ]:
plot_index_map(indices, "90th Percentile Index")

### 10th percentile index

In [ ]:
plot_index_map(indices, "10th Percentile Index")